# Imports

In [ ]:
from canvasapi import Canvas # pip install canvasapi

from autocanvas.config import INPUT_DIR

from autocanvas.core.conversions import (
    series_from_api_object, 
    df_from_api_list)

from autocanvas.core.course_info import (
    get_PHY_course, 
    get_assignment_group_from_name, 
    get_teaching_personel,
    get_sections,
    get_students_from_sections,)

from autocanvas.core.assignments import get_assignment_collection

from os.path import join
import re
import pandas as pd     # manipulate tabulated data
pd.set_option('display.max_columns', 500)
from datetime import datetime 


API_URL = "https://ufl.instructure.com/"
try:
    from autocanvas.config import get_API_key
    API_KEY = get_API_key()
except FileNotFoundError as e:
    print(e)
    API_KEY = input("Asking for API token")

canvas = Canvas(API_URL, API_KEY)

# Get regular quizzes

In [ ]:
course = get_PHY_course(canvas, course_code="PHY2054", semester="Spring 2021")
print(course.name)

In [ ]:
df_regular_quizzes = get_assignment_collection(course,
                             assignment_group_name="recitation quizzes",
                             name_pattern="^Quiz \d+",
                             exclude_numbers=[0,10,13],
                             add_identifier_numbers=True
                            )
df_regular_quizzes

# Import Section Info from spreadsheet

Even if we eventually are able to get all this info directly from the registrar website, still it would be good to store the data in a similar spreadsheet, so that we download it only once.

In [ ]:
excel_fname = "quiz_times_spring2021.xlsx"
excel_path = join(INPUT_DIR, excel_fname)

In [ ]:
# TODO: skipfooter should adjust to the number of the DRC students
df_excel = (pd.read_excel(excel_path, skiprows=19,
                        skipfooter=59, convert_float=True)
                    .set_index("Class #")
                    .filter(regex='^Quiz \d+ (start|due) time$')
                    .dropna(axis=1, how='any')
                    .applymap(lambda x: str(x.round("min")) )
        )
print(len(df_excel))
df_excel.head(50) # 0change to 50 in order to check all sections

In [ ]:
df_sections_api = get_sections(course,add_TA_info=False,remove_empty=True)
print(len(df_sections_api))

In [ ]:
# df_sections_api

In [ ]:
# how==right schedules only non-empty sections
# how==left schedules every section in excel
# always check that the number of sections in 
# df_full is what you want

how="right"
df_full = pd.merge(left=df_excel, right=df_sections_api, how=how, 
                  left_index=True, right_on="class_number", validate="1:1")
df_full = df_full[ ["class_number"] + [ col for col in df_full.columns if col != "class_number" ] ]
print(len(df_full))
df_full.head(50)

In [ ]:
# sections = course.get_sections()
for row_id, assignment in df_regular_quizzes.iterrows():
    # Initial checks
#     assert assignment.name=="Test quiz", "Check everything before you disable safeguard" 
    print("\n", assignment["name"], "\n")
    assert len(assignment["name"].split()) == 2, "Expecting format `Quiz/Exam ###`"
    
    quiz_number = assignment["identifier_number"]
    column_start = "Quiz {} start time".format(quiz_number)
    column_due = "Quiz {} due time".format(quiz_number)
    assignment_object = assignment["object"]
    
    for section_id, section in df_full.iterrows():
        
        # the API accepts both the date format in excel and datetime object
        datetime_start = datetime.fromisoformat(section[column_start])
        datetime_due = datetime.fromisoformat(section[column_due])
        now = datetime.now()
        
        # making sure that the assignment is in the future
        # TODO: make it robust over timezones
        if datetime_start < now:
            print("Skipping Section: {}. Requested start time ({}) is in the past."
                     .format(section["class_number"],datetime_start.isoformat()))
            continue
        try:
            assignment_object.create_override(
                assignment_override={
                "course_section_id":section_id,
                "title":section["name"],
                "due_at":datetime_due,
                "unlock_at":datetime_start,
                "lock_at":datetime_due
                })
            print(section["class_number"])
        except Exception as e:
            if '"message":"taken"' in str(e):
                print("Skipping Section: {}. Override already present."
                                    .format(section["class_number"]))
            else:
                print(e)
            